In [2]:
import numpy as np
import os
import h5py, pickle
from itertools import permutations
from sympy.combinatorics import Permutation

cfg2old=lambda cfg: cfg[1:]+'_r'+{'a':'0','b':'1','c':'2','d':'3'}[cfg[0]]
cfg2new=lambda cfg: {'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]] + cfg[:4]
projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
inds_trace=[i for i,ins in enumerate(inserts) if ins[0]==ins[1]]

xyzt2xyz0=lambda x: x if x!='t' else '0'
t=[insert for insert in inserts]; inserts_key=[f'=der:g{xyzt2xyz0(insert[1])}D{xyzt2xyz0(insert[0])}:sym=' for insert in t]

elements=[(sx,sy,sz,xyz) for sx in [1,-1] for sy in [1,-1] for sz in [1,-1] for xyz in permutations([0, 1, 2], 3)] # Permute first Flip next
def rotate_mom(e,mom):
    sx,sy,sz,xyz=e; ix,iy,iz=xyz; iix,iiy,iiz=tuple([ix,iy,iz].index(i) for i in range(3))
    return [sx*mom[iix],sy*mom[iiy],sz*mom[iiz],sx*mom[iix+3],sy*mom[iiy+3],sz*mom[iiz+3]]
def rotate_vec3(e,vec3): #xyzt=0123
    if vec3 in ['t']:
        return (1,vec3)
    sx,sy,sz,xyz=e; ivec3={'x':0,'y':1,'z':2}[vec3]; ivec3_new=xyz[ivec3]; vec3_new=['x','y','z'][ivec3_new]
    sign=[sx,sy,sz][ivec3_new]
    return (sign,vec3_new)
def rotate_proj(e,proj):
    if proj=='P0':
        return (1,proj)
    sx,sy,sz,xyz=e; det=sx*sy*sz*(1 if Permutation(xyz).is_even else -1)
    (sign,proj_new)=rotate_vec3(e,proj[-1])    
    return (sign*det,proj_new)
def rotate_insert(e,insert):
    s1,i1=rotate_vec3(e,insert[0]); s2,i2=rotate_vec3(e,insert[1])
    return (s1*s2,i1+i2 if i1+i2 in inserts else i2+i1)

def mom2moms(mom):
    moms=list(set([tuple(rotate_mom(e,mom)) for e in elements])) 
    moms.sort()
    moms=np.array(moms)
    return moms
def mom2standard(mom):
    moms=list(set([tuple(rotate_mom(e,mom)) for e in elements])) 
    return list(max(moms, key=lambda x: x[::-1]))

def mom2name(mom):
    assert(np.all(mom==mom2standard(mom)))
    return ','.join([str(ele) for ele in mom])

In [4]:
avgmoreQ=True
def avgmore(dat,mom):
    if avgmoreQ:
        assert(np.all(mom==mom2standard(mom)))
        moms=mom2moms(mom)
        dic={}
        for i,m in enumerate(moms):
            dic[tuple(m)]=i

        e2ind_mom={}; e2inds_proj={}; e2signs_proj={}; e2inds_insert={}; e2signs_insert={}
        for e in elements:
            e2ind_mom[e]=dic[tuple(rotate_mom(e,mom))]
            
            sx,sy,sz,xyz=e; signs=[sx,sy,sz,1]
            ix,iy,iz=xyz; iix,iiy,iiz=tuple([ix,iy,iz].index(i) for i in range(3))
            xyzt=['x','y','z','t']
            xyzt2={'x':xyzt[ix],'y':xyzt[iy],'z':xyzt[iz],'t':'t'}
            det=sx*sy*sz*(1 if Permutation(xyz).is_even else -1)
            
            e2signs_proj[e]=np.array([1,sx*det,sy*det,sz*det])
            e2inds_proj[e]=[0,ix+1,iy+1,iz+1]
            
            e2signs_insert[e]=np.array([signs[xyzt.index(insert[0])]*signs[xyzt.index(insert[1])] for insert in inserts])
            e2inds_insert[e]=[xyzt2[insert[0]]+xyzt2[insert[1]] for insert in inserts]
            e2inds_insert[e]=[inserts.index(ele) if ele in inserts else inserts.index(ele[1]+ele[0]) for ele in e2inds_insert[e]]
            
        def get(e):
            t=dat[:,:,e2ind_mom[e]]
            
            t=t*e2signs_proj[e][None,None,:,None]
            t=t[:,:,e2inds_proj[e]]
            
            t=t*e2signs_insert[e][None,None,None,:]
            t=t[:,:,:,e2inds_insert[e]]
            
            return t
        dat=np.mean([get(e) for e in elements],axis=0)[:,:,None,:,:]
        
    return dat

mom=[0,1,1,0,0,0]

path='/p/project1/ngff/li47/code/projectData/02_discNJN_1D/data_conn/B64_2units_conn.h5'
outpath='/p/project1/ngff/li47/code/projectData/05_moments/cB211.072.64/data_merge/conn_0,1,1,0,0,0_alpha1Run.h5'
with h5py.File(path) as f, h5py.File(outpath,'w') as fw:
    cfgs=f['cfgs'][:]
    fw.create_dataset('notes',data=['cfg,time,mom,proj,insert','mom=[sink,ins]; sink+ins=src','proj=[P0,Px,Py,Pz]'])
    fw.create_dataset('inserts',data=inserts)
    fw.create_dataset('cfgs',data=cfgs)
    fw.create_dataset('moms',data=[mom] if avgmoreQ else mom2moms(mom))
    
    moms=mom2moms(mom)
    moms_old=f['moms'][:]
    dic={}
    for i,m in enumerate(moms_old):
        dic[tuple(list(m)+[0,0,0])]=i
    inds_moms=[dic[tuple(m)] for m in moms]
    for key in f['data'].keys():
        t=f['data'][key][:]
        t=t[:,:,inds_moms]
        t=avgmore(t,mom)
        
        fw.create_dataset(f'data/{key}',data=t)

In [5]:
path='/p/project1/ngff/li47/code/projectData/02_discNJN_1D/data_conn/B64_2units_Ncorr.h5'
outpath='/p/project1/ngff/li47/code/projectData/05_moments/cB211.072.64/data_merge/conn_2pt_alpha1Run.h5'
with h5py.File(path) as f, h5py.File(outpath,'w') as fw:
    fw.create_dataset('notes',data=['cfg,time,mom'])
    fw.create_dataset('inserts',data=inserts)
    fw.create_dataset('cfgs',data=cfgs)
    fw.create_dataset('moms',data=[mom])
    
    for key in f.keys():
        t=np.real(f[key][:])
        fw.create_dataset(f'data/{key}',data=np.real(t[:,:,None]))